# Blast for metabarcoding

### Looking at the reference database

In [1]:
!head ../Data/Bat_Data/reference_fasta.fas

>GBACU292-12|Tachysphex pompiliformis|COI-5P|MH611210
---------------------------------------------------AAGTTTAATTATTCGAATAGAATTAGGTACAGCCGGACAAGTTATTGGTAATGATCAGATTTATAATTCAATAGTAACGGCTCATGCTTTTGTTATAATTTTTTTTATAGTTATGCCTTTTATAATTGGTGGGTTTGGAAATTGATTAATTCCATTAATATTAGGATCACCTGATATAGCTTTTCCTCGAATAAATAATATAAGATTTTGATTATTGCCCCCTTCTATTATTATGTTAATTTTAGGTAATGTTTTTGATTCAGGGGTTGGTACCGGGTGGACAATATATCCTCCATTATCTTCAAATTTAGGACATATAGGATGTTCTGTTGATATTTCTATTTTTTCTTTACATATTGCTGGTATTTCTTCTATTATAGGTGCTATTAATTTTATTGTAACTATTAAAAATATGGTTAATTTAGGTTTATTATTTGATCAAGTTCCATTGTTTGTTTGGTCTGTAATAATTACTGCAATTTTATTATTATTATCATTACCTGTTTTAGCTAGAGCTATTACAATATTATTAACTGATCGTAATTTAAATAC-------------------------------------------------------
>GBACU2922-13|Tachysphex obscuripennis|COI-5P|MH609445
TATATTATATTTTTTATTTGCTATATGATCAGGAATATTAGGGTCATCTTTAAGATTAATTGTACGAATAGAATTAGGAACGGCTGGTCAAGTAATTGGTAATGATCAAATTTATAATTCAATTGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTTTTATAATTGGAGGGTTTGGTAATTGATTAATTCCTTTAATATTAGGATCACCAGATATGGCA

Let's replace the space in between the genus and species line on our BOLD reference database in order to get the full header to print in the tabulated datafile

In [7]:
header = "GBACU2923-13|Tachysphex obscuripennis|COI-5P|MH611205"
header

'GBACU2923-13|Tachysphex obscuripennis|COI-5P|MH611205'

In [13]:
!echo "${header}" | sed -e 's/ /_/g' 

-13|Tachysphex_obscuripennis|COI-5P|MH611205


In [17]:
!sed -e 's/ /_/g' ../Data/Bat_Data/reference_fasta.fas > ../Data/Bat_Data/reference_under.fas

In [18]:
!head ../Data/Bat_Data/reference_under.fas

>GBACU292-12|Tachysphex_pompiliformis|COI-5P|MH611210
---------------------------------------------------AAGTTTAATTATTCGAATAGAATTAGGTACAGCCGGACAAGTTATTGGTAATGATCAGATTTATAATTCAATAGTAACGGCTCATGCTTTTGTTATAATTTTTTTTATAGTTATGCCTTTTATAATTGGTGGGTTTGGAAATTGATTAATTCCATTAATATTAGGATCACCTGATATAGCTTTTCCTCGAATAAATAATATAAGATTTTGATTATTGCCCCCTTCTATTATTATGTTAATTTTAGGTAATGTTTTTGATTCAGGGGTTGGTACCGGGTGGACAATATATCCTCCATTATCTTCAAATTTAGGACATATAGGATGTTCTGTTGATATTTCTATTTTTTCTTTACATATTGCTGGTATTTCTTCTATTATAGGTGCTATTAATTTTATTGTAACTATTAAAAATATGGTTAATTTAGGTTTATTATTTGATCAAGTTCCATTGTTTGTTTGGTCTGTAATAATTACTGCAATTTTATTATTATTATCATTACCTGTTTTAGCTAGAGCTATTACAATATTATTAACTGATCGTAATTTAAATAC-------------------------------------------------------
>GBACU2922-13|Tachysphex_obscuripennis|COI-5P|MH609445
TATATTATATTTTTTATTTGCTATATGATCAGGAATATTAGGGTCATCTTTAAGATTAATTGTACGAATAGAATTAGGAACGGCTGGTCAAGTAATTGGTAATGATCAAATTTATAATTCAATTGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTTTTATAATTGGAGGGTTTGGTAATTGATTAATTCCTTTAATATTAGGATCACCAGATATGGCA

Look at ncbi reference in comparison

In [1]:
!head ../Data/Bat_Data/sequence.fasta

>KF396611.1 'EulechriaX' sciaphila voucher CCDB-15792-C01 cytochrome oxidase subunit 1 (COI) gene, partial cds; mitochondrial
ACTATTGTTACAGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTG
GTAATTGATTAGTACCTTTAATATTAGGAGCCCCAGATATAGCTTTCCCACGAATAAATAATATAAGATT
TTGACTTTTACCCCCCTCATTAACTTTATTAATTTCAAGAAGAATTGTAGAAAATGGGGCTGGAACTGGT
TGAACAGTTTATCCACCTTTATCTTCTAATATTGCTCATAGAGGTAGTTCAGTTGATTTAGCAATTTTCT
CCCTTCACTTAGCTGGTATTTCATCAATTTTAGGGGCTATTAATTTTATTACAACAATTATTAATATACG
ACTTGGAAATATATCTTTTGATCAAATACCTTTATTTGTTTGATCAGTAGGTATTACAGCTTTATTATTA
TTATTATCTCTTCCAGTATTAGCTGGGGCTATTACTATATTACTTACAGATCGAAATCTTAATACTTCTT
TCTTTGATCCTGCAGGAGGAGGAGATCCTATTCTTTATCAACACTTATTT



# Blast Time

Using this framework for our samples: https://docs.google.com/document/d/1h9d0JrTsDLzsOV5klMkD47807dWTmcXN3uxoYp0ei64/edit#heading=h.mli2uhakn4sx

See section 4.1 for more details

### Check that blast is here and ready to use

In [1]:
bldir = "/Applications/ncbi-blast-2.11.0+/bin/"
bldir

'/Applications/ncbi-blast-2.11.0+/bin/'

In [3]:
!ls /Applications/ncbi-blast-2.11.0+/bin

blast_formatter            legacy_blast.pl
blastdb_aliastool          makeblastdb
blastdbcheck               makembindex
blastdbcmd                 makeprofiledb
blastn                     psiblast
blastp                     rpsblast
blastx                     rpstblastn
cleanup-blastdb-volumes.py segmasker
convert2blastmask          tblastn
deltablast                 tblastx
dustmasker                 update_blastdb.pl
get_species_taxids.sh      windowmasker


In [2]:
!{bldir}makeblastdb \
-in ../Data/Bat_Data/reference_under.fas \
-dbtype nucl \
-out ../Data/Bat_Data/reference




Building a new DB, current time: 02/26/2021 07:59:21
New DB name:   /Users/tracykreling/Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data/reference
New DB title:  ../Data/Bat_Data/reference_under.fas
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/tracykreling/Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data/reference
Keep MBits: T
Maximum file size: 1000000000B
FASTA-Reader: Ignoring invalid residues at position(s): On line 6427678: 1-2
FASTA-Reader: Ignoring invalid residues at position(s): On line 9302272: 660
Adding sequences from FASTA; added 5425365 sequences in 149.749 seconds.




## Take a look at the files

Let's take a look at the reference file

In [3]:
! head ../Data/Bat_Data/reference_under.fas

>GBACU292-12|Tachysphex_pompiliformis|COI-5P|MH611210
---------------------------------------------------AAGTTTAATTATTCGAATAGAATTAGGTACAGCCGGACAAGTTATTGGTAATGATCAGATTTATAATTCAATAGTAACGGCTCATGCTTTTGTTATAATTTTTTTTATAGTTATGCCTTTTATAATTGGTGGGTTTGGAAATTGATTAATTCCATTAATATTAGGATCACCTGATATAGCTTTTCCTCGAATAAATAATATAAGATTTTGATTATTGCCCCCTTCTATTATTATGTTAATTTTAGGTAATGTTTTTGATTCAGGGGTTGGTACCGGGTGGACAATATATCCTCCATTATCTTCAAATTTAGGACATATAGGATGTTCTGTTGATATTTCTATTTTTTCTTTACATATTGCTGGTATTTCTTCTATTATAGGTGCTATTAATTTTATTGTAACTATTAAAAATATGGTTAATTTAGGTTTATTATTTGATCAAGTTCCATTGTTTGTTTGGTCTGTAATAATTACTGCAATTTTATTATTATTATCATTACCTGTTTTAGCTAGAGCTATTACAATATTATTAACTGATCGTAATTTAAATAC-------------------------------------------------------
>GBACU2922-13|Tachysphex_obscuripennis|COI-5P|MH609445
TATATTATATTTTTTATTTGCTATATGATCAGGAATATTAGGGTCATCTTTAAGATTAATTGTACGAATAGAATTAGGAACGGCTGGTCAAGTAATTGGTAATGATCAAATTTATAATTCAATTGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTTTTATAATTGGAGGGTTTGGTAATTGATTAATTCCTTTAATATTAGGATCACCAGATATGGCA

Let's take a look at the data file. Here are the first few entries of the forward and reverse reads for the second sample. We want to make sure that the flowcells are in the same order. They appear to be, so we should be able to merge the reads correctly using *cat*

In [7]:
!head ../Data/Bat_Data/2_S2_L001_R1_001.fastq

@M01998:2:000000000-A5JRV:1:1101:16769:1826 1:N:0:2
TACAGATATTGGAACTTTATATTTTATTTTTGGTATTTGAGCTGGTATAGTTGGAGCCTCTTTAAGATTACTAATTCGAGCTGAATTAGGAACCCCAGGATCTTTAATTGGAGATGATCAAATTTATAATACTATTGTAACAGCACATGCTTTTATTATAATTTTTTTCATAGTTATACCTATTATAATCGGAGGATTTGGTAATTGATTAGTAGCA
+
A3A3AFDFFFD44BEGGFFDFGHHHHHHHHHEGGGHHHGCGFHFGFGFDHHHHGBFEEHHHHHHGFHHHHHHHHHHHEFGFGEFHHHEE55EFFGG?EEEGHHHHFHHHF3BBGFFFHHGHGGHHHHGHHHHHHHGHHGHHEFGFGHHGHHHHHHHFHHGGGHHHHGGGHHHHHHHHHHFHFHHHHHFHH?A//-CGHHGHHHHHFHFHHGFHGEHB
@M01998:2:000000000-A5JRV:1:1101:15533:1832 1:N:0:2
TTAGATATTGGAACATTATATTTTATTTTTGGTATTTGAGCTGGAATAGTAGGGACTTCTTTAAGACTATTAATTCGGGCTGAATTAGGAAATCCAGGATCTTTAATTGGAGATGATCAAATTTATAATACTATTGTTACAGCACATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGTAATTGATTAGTAAC
+
BBBA@DDFFBFDGGBFGFDFGGHHGHHHHHCGHHHHHBGDHGBBGHHCGGFBGGFFEGGFHHHHHGHHHHHEFGHGFGDGE?GGFHFFHHFHFFHBFGEHFDHHFFHHGHECFGBHHGHHHFHHFH4GHGHFGHH3GGHH4BGFEHGFHFGHHFHHFHFGHHHHHGGGHHFGGFFFFHHHHHH1GDDGHDFGBDDGHG1FGFHGHFBFGFFHHBF
@M01998:2:000000000-A5JR

In [8]:
!head ../Data/Bat_Data/2_S2_L001_R2_001.fastq

@M01998:2:000000000-A5JRV:1:1101:16769:1826 2:N:0:2
TGCTACTAATCAATTACCAAATCCTCCGATTATAATAGGTATAACTATGAAAAAAATTATAATAAAAGCATGTGCTGTTACAATAGTATTATAAATTTGATCATCTCCAATTAAAGATCCTGGGGTTCCTAATTCAGCTCGAATTAGTAATCTTAAAGAGGCTCCAACTATACCAGCTCAAATACCAAAAATAAAATATAAAGTTCCAATATCTGTA
+
A>AABFFFFFFFGGGGGGGGGGHHHHHGCGFGDGGHFHHAFHFHHFAGFH5F5EEGGHFFGFHFF55FGHHFHHHHFHHHHHHHHHHFGHGHHGFHHHGHHHFHHHHHHHGHHGFHEHHHHF3FCGGHFHHFGHHGFFHHC?FGGDHHHHHHHFFG33B2?AFGE<GFHHHFFHBGHFHHBFDFHGFHHEGGHHFFFHG11FGG<GGHGDGGHD0=D
@M01998:2:000000000-A5JRV:1:1101:15533:1832 2:N:0:2
GTTACTAATCAATTACCAAATCCTCCAATTATAATAGGTATAACTATAAAAAAAATTATAATAAAAGCATGTGCTGTAACAATAGTATTATAAATTTGATCATCTCCAATTAAAGATCCTGGATTTCCTAATTCAGCCCGAATTAATAGTCTTAAAGAAGTCCCTACTATTCCAGCTCAAATACCAAAAATAAAATATAATGTTCCAATATCTAA
+
3>AABFFFFFFFFGFFGGGGGGHFFHFFHHFHHHHGBGHHHGHHFFFHHHHHEGGFHG5GGFGFGGFHFHFHHHHHGFGHHHHHHHGHHHGFGFGHHHGGHGHGBGHHHHHFHHGHHGHBGFHDHHHHHGHFF4BBG3EECGEHFDDGFEGFGHHHBFH2HGBGGFHGFGHHF2FGGFHFFHGEH1GHHHHHHHHHGGBDFFGGFGHG1<DD<11
@M01998:2:000000000-A5JR

### Trim Samples

In [ ]:
! seqtk trimfq ../Data/Bat_Data/*.fastq > ../Data/Analyzed/*trimmed_.fastq

In [11]:
! seqtk trimfq ../Data/Bat_Data/2_S2_L001_R2_001.fastq > ../Data/Bat_Data/2_S2_L001_R2_001_trimmed.fastq

In [13]:
! seqtk trimfq ../Data/Bat_Data/2_S2_L001_R1_001.fastq > ../Data/Bat_Data/2_S2_L001_R1_001_trimmed.fastq

### Convert to fasta

In [14]:
! seqtk seq -a  ../Data/Bat_Data/2_S2_L001_R1_001_trimmed.fastq > ../Data/Bat_Data/2_S2_L001_R1_001_trimmed.fasta

In [15]:
! seqtk seq -a  ../Data/Bat_Data/2_S2_L001_R2_001_trimmed.fastq > ../Data/Bat_Data/2_S2_L001_R2_001_trimmed.fasta

In [1]:
!head ../Data/Bat_Data/2_S2_L001_R1_001_trimmed.fasta

>M01998:2:000000000-A5JRV:1:1101:16769:1826 1:N:0:2
TACAGATATTGGAACTTTATATTTTATTTTTGGTATTTGAGCTGGTATAGTTGGAGCCTCTTTAAGATTACTAATTCGAGCTGAATTAGGAACCCCAGGATCTTTAATTGGAGATGATCAAATTTATAATACTATTGTAACAGCACATGCTTTTATTATAATTTTTTTCATAGTTATACCTATTATAATCGGAGGATTTGGTAATTGATTAGTAGCA
>M01998:2:000000000-A5JRV:1:1101:15533:1832 1:N:0:2
TTAGATATTGGAACATTATATTTTATTTTTGGTATTTGAGCTGGAATAGTAGGGACTTCTTTAAGACTATTAATTCGGGCTGAATTAGGAAATCCAGGATCTTTAATTGGAGATGATCAAATTTATAATACTATTGTTACAGCACATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGTAATTGATTAGTAAC
>M01998:2:000000000-A5JRV:1:1101:18960:1841 1:N:0:2
GTAGATATTGGAACATTATATTTTATTTTTGGTATTTGAGCTGGAATAGTAGGAACTTCTTTAAGACTATTAATTCGGGCTGAATTAGGAAATCCAGGATCTTTAATTGGAGATGATCAAATTTATAATACTATTGTTACAGCACATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGTAATTGATTAGTTAC
>M01998:2:000000000-A5JRV:1:1101:18978:1853 1:N:0:2
GTAGATATTGGAACATTATATTTTATTTTTGGTATTTGAGCTGGAATAGTAGGAACTTCTTTAAGACTATTAATTCGGGCTGAATTAGGAAATCCAGGATCTTTAATTGGAGATGATCAAATTTATAATACTATTGTTACAG

## Blastn

*NOTE*: there is something wrong with the first sample forward read and blast won't work with it. May need to redownload and see if that helps in case I accdientally did something to the original file at some point. Good news is the second sample appears to be working pretty well. Actually on second though, it appears to be working and just giving a warning with that file... That was prior to trimming so maybe there were just some really bad calls at the end there and that was throwing it off. I'll try again with a trimmed file and see what happens

In [2]:
!head ../Data/Bat_Data/2_S2_L001_R1_001_trimmed_reference_blastn.tab

In [ ]:
!{bldir}blastn \
-db ../Data/Bat_Data/reference \
-query ../Data/Bat_Data/2_S2_L001_R2_001_trimmed.fasta \
-out ../Results/Bat/2_S2_L001_R2_001_trimmed_referenceu_blastn.tab \
-outfmt 6 


In [ ]:
!head ../Results/Bat/2_S2_L001_R2_001_trimmed_reference_blastn.tab

In [36]:
!head ../Results/Bat/1_S1_L001_R1_001_reference_blastn.tab

M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKD545-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKD237-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKB702-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKA760-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKA707-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKA410-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKA237-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKA020-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKA629-09|Agrotis	99.010	202	2	0	13	214	1	202	3.29e-98	363
M01998:2:000000000-A5JRV:1:1101:14207:1817	CGUKA373-09|Agrotis	9